In [5]:
import numpy as np
import scipy.sparse as sp

from src.algorithms import UserBasedCF, ItemBasedCF, CBF
from src.recommender_model import RecommenderModel
from src.utils import tf_idf, train_model, write_submission

# Hybrid Model: Content-Based Filtering and Collaborative Filtering
This notebook provides a simple hybrid model taking advantage of several filtering methods that uses similarity between users/items/features.

In [7]:
class HybridCBFCF(RecommenderModel):
	def __init__(self, ubcf_coeff: float, ibcf_coeff: float, cbf_coeff: float):
		super(HybridCBFCF, self).__init__()
		self.ubcf = UserBasedCF()
		self.ibcf = ItemBasedCF()
		self.cbf = CBF()

		self.ubcf_coeff = ubcf_coeff
		self.ibcf_coeff = ibcf_coeff
		self.cbf_coeff = cbf_coeff

	def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix) -> None:
		self.urm = urm
		self.icm = icm

		self.ubcf.fit(urm, icm)
		self.ibcf.fit(urm, icm)
		self.cbf.fit(urm, icm)

	def _get_recommendations_predictions(self, user_id: int) -> np.ndarray:
		return (
			self.ubcf_coeff * self.ubcf._get_recommendations_predictions(user_id) +
			self.ibcf_coeff * self.ibcf._get_recommendations_predictions(user_id) +
			self.cbf_coeff * self.cbf._get_recommendations_predictions(user_id)
        )

In [ ]:
hybrid_cbf_cb = train_model(HybridCBFCF(1, .5, .2))

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 35736 (100.0%), 503.68 column/sec. Elapsed time 1.18 min
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 549.96 column/sec. Elapsed time 1.16 min
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 384.54 column/sec. Elapsed time 1.65 min


In [ ]:
hybrid_cbf_cb_submission = train_model(HybridCBFCF(1, .5, .2), test_size=0)
write_submission(hybrid_cbf_cb_submission, "hybrid_cbf_cb_submission.csv")